In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import glob

# See "Diseases.ipynb" for these files
custom = pd.read_excel('./Phenotyping/custom_phenotypes.xlsx')
custom['phecode1.2'] = custom['phecode1.2'].astype(float)
icd = pd.read_csv('./Phenotyping/icd_codes.csv')
merged_pheno = pd.read_excel('./Phenotyping/merged_phenotypes.xlsx')

## MVP

## Genebass

### Exporting data with Hail

In [ ]:
import os
import hail as hl
hl.init(min_block_size=128)
import pandas as pd
!export PYSPARK_SUBMIT_ARGS="--driver-memory 144g --executor-memory 144g pyspark-shell"

pt = hl.read_table('/sc/arion/projects/data-ark/Public_Unrestricted/Genebass/pheno_results.ht')
df = pt.to_pandas()
df.to_excel('./Genebass/pheno_results.xlsx')


In [2]:
gb = hl.read_matrix_table('/sc/arion/projects/data-ark/Public_Unrestricted/Genebass/results.mt')
gb = gb.drop('markerAFs','n_cases','n_controls','heritability','saige_version','inv_normalized','n_cases_defined',
             'n_cases_both_sexes','n_cases_males','n_cases_females','coding_description','description_more',
             'interval','markerIDs','Nmarker_MACCate_1','Nmarker_MACCate_2','Nmarker_MACCate_3',
             'Nmarker_MACCate_4','Nmarker_MACCate_5','Nmarker_MACCate_6','Nmarker_MACCate_7','Nmarker_MACCate_8',
             'Pvalue.NA','Pvalue_Burden.NA','Pvalue_SKAT.NA','BETA_Burden.NA','SE_Burden.NA',
             'Pvalue','Pvalue_SKAT')
gb = gb.filter_cols(
    (gb.trait_type == 'icd10') | (gb.trait_type == 'icd_first_occurrence') | ((gb.modifier == 'custom') & (gb.trait_type == 'categorical'))
)
gb = gb.filter_rows((gb.annotation != 'synonymous'))

gb = gb.filter_entries((gb.Pvalue_Burden < 0.05))

gb = gb.key_rows_by()
gb = gb.key_cols_by() 
gb = gb.drop('pheno_sex','trait_type','coding','modifier')

gb = gb.entries()
gb.export('/sc/arion/projects/GENECAD/Robert/DOE/Genebass/GB/gb.tsv')


SLF4J: Failed to load class "org.slf4j.impl.StaticMDCBinder".
SLF4J: Defaulting to no-operation MDCAdapter implementation.
SLF4J: See http://www.slf4j.org/codes.html#no_static_mdc_binder for further details.
2024-12-14 14:44:20.249 Hail: INFO: merging 1001 files totalling 374.9M... 1000]
2024-12-14 14:44:20.672 Hail: INFO: while writing:
    /sc/arion/projects/GENECAD/Robert/DOE/Genebass/GB/gb.tsv
  merge time: 422.232ms


In [ ]:
mt = hl.read_matrix_table('/sc/arion/projects/data-ark/Public_Unrestricted/Genebass/variant_results.mt')
mt = mt.drop('heritability','saige_version','inv_normalized',
             'n_cases_both_sexes','n_cases_females','n_cases_males','coding_description','description_more',
             'coding_description','description','n_cases_defined','category',
             'call_stats','AF.Cases','AF.Controls')
mt = mt.filter_cols(
    (mt.trait_type == 'icd10') | (mt.trait_type == 'icd_first_occurrence') | ((mt.modifier == 'custom') & (mt.trait_type == 'categorical'))
)
mt = mt.filter_rows((mt.annotation != 'synonymous'))

mt = mt.filter_entries((mt.Pvalue < 0.05))

mt = mt.key_rows_by()
mt = mt.key_cols_by() 
mt = mt.drop('locus','alleles','pheno_sex','trait_type','coding','modifier')

mt = mt.entries()
mt.export('/sc/arion/projects/GENECAD/Robert/DOE/Genebass/SV/sv.tsv')


### Annotating variants with VEP

#### Running VEP with missense scoring

In [ ]:
df = pd.read_csv('/sc/arion/projects/GENECAD/Robert/DOE/Genebass/SV/sv.tsv', sep='\t')
df.to_pickle('./Genebass/SV/sv.pkl')
df['markerID'].drop_duplicates().to_pickle('./Genebass/SV/sv_unique.pkl')

In [ ]:
df = pd.read_pickle('./Genebass/SV/sv_unique.pkl')
df = pd.DataFrame(df)
df['chr'] = df['markerID'].str.split(':').str[0]
df['pos'] = df['markerID'].str.split(':').str[1].str.split('_')
df['ref'] = df['pos'].str[1]
df['pos'] = df['pos'].str[0].astype(int)
df['ref'] = df['ref'].str.split('/')
df['alt'] = df['ref'].str[1]
df['ref'] = df['ref'].str[0]
df['chr'] = df['chr'].str.replace('chr','')
df = df.reset_index(drop=True)
df = df.rename({'markerID':'id'},axis=1)

# Add necessary VCF fields
df['qual'] = '.'
df['filter'] = '.'
df['info'] = '.'

# Reorder columns to VCF format
vcf_columns = ['chr', 'pos', 'id', 'ref', 'alt', 'qual', 'filter', 'info']
vcf_df = df[vcf_columns]
vcf_df = vcf_df.sort_values(['chr','pos','ref','alt'])

# Save as VCF in chunks for VEP annotation
vcf_header = """##fileformat=VCFv4.2
##source=CustomGeneratedVCF
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO
"""
with open('./Genebass/SV/sv_unique.vcf', 'w') as f:
    f.write(vcf_header)
    vcf_df.to_csv(f, sep='\t', index=False, header=False)

chunk_size = 400000
num_chunks = len(vcf_df) // chunk_size + (len(vcf_df) % chunk_size > 0)

for i in range(num_chunks):
    chunk_file = f'./Genebass/SV/sv_unique_chunk_{i+1}.vcf'
    start = i * chunk_size
    end = start + chunk_size
    chunk = vcf_df.iloc[start:end]
    
    with open(chunk_file, 'w') as f:
        f.write(vcf_header)
        chunk.to_csv(f, sep='\t', index=False, header=False)
        

In [ ]:
for chunk in range(1,15):
    vcf = pd.read_csv(f'./Genebass/SV/sv_unique_chunk_{chunk}_vep_filter.vcf', sep='\t', skiprows=53)
    vcf = vcf.set_axis(['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'],axis=1).drop(['QUAL','FILTER'],axis=1)
    vcf = vcf.assign(INFO=vcf['INFO'].str.split(',')).explode('INFO')
    vcf['INFO'] = vcf['INFO'].str.split('|')
    
    column_names = [
        'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature',
        'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position',
        'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'REF_ALLELE',
        'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT',
        'Aloft_pred', 'AlphaMissense_pred', 'AlphaMissense_rankscore', 'AlphaMissense_score',
        'BayesDel_addAF_pred', 'BayesDel_noAF_pred', 'CADD_raw', 'CADD_raw_rankscore',
        'ClinPred_pred', 'DANN_rankscore', 'DANN_score', 'DEOGEN2_pred', 'Eigen-PC-raw_coding',
        'Eigen-PC-raw_coding_rankscore', 'Eigen-raw_coding', 'Eigen-raw_coding_rankscore',
        'FATHMM_pred', 'LIST-S2_pred', 'LRT_pred', 'M-CAP_pred', 'MPC_rankscore', 'MPC_score',
        'MVP_rankscore', 'MVP_score', 'MetaLR_pred', 'MetaSVM_pred', 'MutPred_rankscore',
        'MutPred_score', 'MutationAssessor_pred', 'MutationTaster_pred', 'PROVEAN_pred',
        'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 'PrimateAI_pred', 'REVEL_rankscore',
        'REVEL_score', 'SIFT4G_pred', 'SIFT_pred', 'VEST4_rankscore', 'VEST4_score',
        'fathmm-MKL_coding_pred', 'fathmm-MKL_coding_rankscore', 'fathmm-XF_coding_pred',
        'fathmm-XF_coding_rankscore', 'gnomAD_exomes_AF', 'gnomAD_exomes_POPMAX_AF',
        'gnomAD_genomes_AF', 'gnomAD_genomes_POPMAX_AF'
    ]
    
    vcf[column_names] = pd.DataFrame(vcf['INFO'].tolist(), index=vcf.index)
    vcf = vcf.drop(columns=['INFO'])
    vcf = vcf.drop(['Allele','Feature_type','Feature','BIOTYPE','EXON','INTRON','HGVSc','HGVSp',
                    'cDNA_position','CDS_position','Protein_position','Amino_acids','Codons','Existing_variation',
                    'REF_ALLELE','DISTANCE','STRAND','FLAGS','SYMBOL_SOURCE','HGNC_ID',
                    'gnomAD_genomes_AF','gnomAD_genomes_POPMAX_AF'],axis=1)
    
    vcf.loc[vcf['Consequence'].str.contains('stop_lost|start_lost|transcript_amplification|inframe_insertion|inframe_deletion|missense_variant|protein_altering_variant',na=False),'snp_type'] = 'Missense'
    vcf.loc[vcf['Consequence'].str.contains('transcript_ablation|splice_acceptor_variant|splice_donor_variant|stop_gained|frameshift_variant',na=False),'snp_type'] = 'PTV'
    vcf = vcf.loc[vcf['IMPACT'] != 'MODIFIER']
    
    #####
    
    qualitative_tools = ['SIFT_pred', 'SIFT4G_pred', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 
                         'LRT_pred', 'MutationTaster_pred', 'FATHMM_pred', 'PROVEAN_pred', 
                         'MetaSVM_pred', 'MetaLR_pred', 'M-CAP_pred', 'PrimateAI_pred', 
                         'DEOGEN2_pred', 'BayesDel_addAF_pred', 'BayesDel_noAF_pred', 
                         'ClinPred_pred', 'LIST-S2_pred', 'fathmm-MKL_coding_pred', 
                         'fathmm-XF_coding_pred', 'MutationAssessor_pred', 'Aloft_pred']
    
    quantitative_tools = ['VEST4_rankscore', 'REVEL_rankscore', 'MutPred_rankscore', 
                          'MVP_rankscore', 'MPC_rankscore', 'DANN_rankscore', 'CADD_raw_rankscore', 
                          'Eigen-raw_coding_rankscore', 'Eigen-PC-raw_coding_rankscore']
    
    
    def calculate_score(row):
        score = 0
        count = 0
        
        # Qualitative scoring
        for tool in qualitative_tools:
            if tool in row and row[tool] != '':
                if tool == 'MutationAssessor_pred' and row[tool] == 'H':
                    score += 1
                elif tool == 'Aloft_pred' and row[tool] in ['R', 'D']:
                    score += 1
                elif tool == 'MutationTaster_pred' and 'D' in str(row[tool]):
                    score += 1
                elif row[tool] == 'D':
                    score += 1
                count += 1
        
        # Quantitative scoring
        for tool in quantitative_tools:
            if tool in row and row[tool] != '':
                try:
                    if float(row[tool]) > 0.9:
                        score += 1
                    count += 1
                except ValueError:
                    pass  # Ignore non-numeric values
    
        return score, count
    
    # Apply the scoring function
    vcf[['total_score', 'non_missing_count']] = vcf.apply(lambda row: pd.Series(calculate_score(row)), axis=1)
    
    vcf['Pred5'] = 0
    vcf.loc[vcf['SIFT_pred'].str.contains('D',na=False), 'Pred5'] += 1
    vcf.loc[vcf['LRT_pred'].str.contains('D',na=False), 'Pred5'] += 1
    vcf.loc[vcf['MutationTaster_pred'].str.contains('A|D',na=False), 'Pred5'] += 1
    vcf.loc[vcf['Polyphen2_HDIV_pred'].str.contains('D',na=False), 'Pred5'] += 1
    vcf.loc[vcf['Polyphen2_HVAR_pred'].str.contains('D',na=False), 'Pred5'] += 1
    
    #####
    
    vcf = vcf[['ID','CHROM','POS','REF','ALT','snp_type','Consequence','IMPACT','SYMBOL',
               'total_score','non_missing_count','Pred5',
               'AlphaMissense_pred','AlphaMissense_rankscore','AlphaMissense_score',
               'REVEL_rankscore','REVEL_score',
               'gnomAD_exomes_AF','gnomAD_exomes_POPMAX_AF']]
    vcf.to_csv(f'./Genebass/SV/sv_unique_chunk_{chunk}_vep_filter_cleaned.vcf', sep='\t', index=None)

# Combine chunk outputs
file_paths = [f"./Genebass/SV/sv_unique_chunk_{chunk}_vep_filter_cleaned.vcf" for chunk in range(1, 15)]
dfs = [pd.read_csv(file, sep='\t', comment='#') for file in file_paths]
final_df = pd.concat(dfs, ignore_index=True)
final_df.to_pickle('./Genebass/SV/vep_combined.pkl')



#### Running LOFTEE on PTVs

In [ ]:
gbv = pd.read_pickle('./Genebass/SV/vep_combined.pkl')

df = gbv.loc[gbv['snp_type'] == 'PTV']
df = df[['CHROM','POS','ID','REF','ALT']].set_axis(['chr','pos','id','ref','alt'],axis=1)
df['qual'] = '.'
df['filter'] = '.'
df['info'] = '.'

# Reorder columns to VCF format
vcf_columns = ['chr', 'pos', 'id', 'ref', 'alt', 'qual', 'filter', 'info']
vcf_df = df[vcf_columns]
vcf_df = vcf_df.sort_values(['chr','pos','ref','alt'])

# Save as VCF
vcf_header = """##fileformat=VCFv4.2
##source=CustomGeneratedVCF
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO
"""
with open('./Genebass/SV/sv_unique_lof.vcf', 'w') as f:
    f.write(vcf_header)
    vcf_df.to_csv(f, sep='\t', index=False, header=False)
    

In [ ]:
vcf = pd.read_csv(f'./Genebass/SV/sv_unique_lof_loftee_filter.vcf', sep='\t', skiprows=53)
vcf = vcf.set_axis(['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'],axis=1).drop(['QUAL','FILTER'],axis=1)
vcf = vcf.assign(INFO=vcf['INFO'].str.split(',')).explode('INFO')
vcf['INFO'] = vcf['INFO'].str.split('|')

column_names = [
    'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature',
    'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position',
    'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'REF_ALLELE',
    'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT',
    'LoF','LoF_filter','LoF_flags','LoF_info'
]

vcf[column_names] = pd.DataFrame(vcf['INFO'].tolist(), index=vcf.index)
vcf = vcf.drop(columns=['INFO'])

vcf.loc[vcf['Consequence'].str.contains('stop_lost|start_lost|transcript_amplification|inframe_insertion|inframe_deletion|missense_variant|protein_altering_variant',na=False),'snp_type'] = 'Missense'
vcf.loc[vcf['Consequence'].str.contains('transcript_ablation|splice_acceptor_variant|splice_donor_variant|stop_gained|frameshift_variant',na=False),'snp_type'] = 'PTV'
vcf = vcf.loc[vcf['IMPACT'] != 'MODIFIER']
vcf = vcf[['ID','Consequence','SYMBOL','LoF','LoF_filter','LoF_flags','LoF_info']]
vcf.to_pickle('./Genebass/SV/vep_loftee.pkl')


#### Combining VEP files

In [ ]:
vep = pd.read_pickle('./Genebass/SV/vep_combined.pkl')
loftee = pd.read_pickle('./Genebass/SV/vep_loftee.pkl')
vep = vep.merge(loftee, how='left')
vep['missense_score'] = vep['total_score']/vep['non_missing_count']
vep = vep.drop(['LoF_filter','LoF_info'],axis=1)

gof = pd.read_pickle('./LoGoFunc/all_predictions.pkl')
gof = gof.rename({'#CHROM':'CHROM'},axis=1).drop(['ID','prediction','LoGoFunc_LOF','SYMBOL'],axis=1)
vep = vep.merge(gof, how='left')

lof = pd.read_pickle('./LoGoFunc/LOF_predictions.pkl').rename({'#CHROM':'CHROM'},axis=1).drop(['ID','SYMBOL'],axis=1)
vep = vep.merge(lof, how='left')

vep = vep[['ID','CHROM','POS','REF','ALT','snp_type','Consequence','SYMBOL',
           'non_missing_count','missense_score','LoF','LoGoFunc_GOF','LoGoFunc_LOF']]

vep['annotation'] = '8_other_missense'
vep.loc[(vep['non_missing_count'] > 7) & (vep['missense_score'] > 0.5), 'annotation'] = '7_other_missense_0.5'
vep.loc[vep['LoGoFunc_GOF'] > 0, 'annotation'] = '6_gof_missense'
vep.loc[(vep['LoGoFunc_GOF'] > 0) & (vep['non_missing_count'] > 7) & (vep['missense_score'] > 0.5), 'annotation'] = '5_gof_missense_0.5'
vep.loc[vep['LoGoFunc_LOF'] > 0, 'annotation'] = '4_lof_missense'
vep.loc[(vep['LoGoFunc_LOF'] > 0) & (vep['non_missing_count'] > 7) & (vep['missense_score'] > 0.5), 'annotation'] = '3_lof_missense_0.5'
vep.loc[(vep['snp_type'] == 'PTV') | (vep['LoF'] == 'LC'), 'annotation'] = '2_lof_lc'
vep.loc[(vep['LoF'] == 'HC'), 'annotation'] = '1_lof_hc'
vep = vep.sort_values('annotation').drop_duplicates('ID')

vep.loc[vep['LoGoFunc_GOF'] >= 2/3, 'hc_gof'] = 1

vep.to_pickle('./Genebass/SV/vep_cleaned.pkl')


## Jurgens et al. rare variant analysis

In [2]:
results = pd.DataFrame()

masks = ['hclof_noflag_missense0.5_POPMAX0.00001','hclof_noflag_missense0.5_POPMAX0.01',
         'hclof_noflag_POPMAX0.01','missense0.5_POPMAX0.00001',
         'hclof_noflag_missense0.8_POPMAX0.001','missense0.2_POPMAX0.00001',
         'hclof_noflag_missense0.8_POPMAX0.01','hclof_noflag_POPMAX0.001',
         'hclof_noflag_missense0.5_POPMAX0.001']

for mask in masks:
    ss = pd.read_csv(f'./Jurgens/UKB_AoU_MGB_overlapcorrected_cleaned_{mask}.tsv', sep='\t').dropna().set_axis(['gene','phenotype','beta','p'],axis=1)
    ss = ss.loc[ss['p'] < 0.05]
    ss['p'] = -np.log10(ss['p'])
    ss.loc[ss['p'] > 300, 'p'] = 300
    ss['mask'] = mask
    results = pd.concat([results,ss])
    
results = results.rename({'gene':'id'},axis=1)
targets = pd.read_pickle('./OT/Raw/targets.pkl')
targets = targets[['id','approvedSymbol']].rename({'approvedSymbol':'gene'},axis=1)
results = results.merge(targets).drop('id',axis=1)
results['mask'] = results['mask'].str.replace('hclof_noflag_','lof_').str.replace('missense','ms').str.replace('POPMAX','af')
results.to_pickle('./Jurgens/UKB_AoU_MGB_combined.pkl')


## PanUKBB

In [ ]:
pu = pd.read_excel('./PanUKBB/panukbb_pheno.xlsx', usecols = ['trait_type','phenocode','description','aws_link'])

pu_icd = pu.loc[pu['trait_type'] == 'icd10']
pu_icd = pu_icd.loc[(pu_icd['phenocode'].isin(icd['ICD'])) | (pu_icd['phenocode'].isin(custom['icd']))]

pu_p12 = pu.loc[pu['trait_type'] == 'phecode']
pu_p12['phenocode'] = pu_p12['phenocode'].astype(float)
pu_p12 = pu_p12.loc[(pu_p12['phenocode'].isin(pu_p12['Phecode'])) | (pu_p12['phenocode'].isin(custom['phecode1.2']))]

pu = pd.concat([pu_icd,pu_p12])

for i, chunk in enumerate(range(0, len(pu), 100)):
    pu['aws_link'].iloc[chunk:chunk+100].to_csv(f'/sc/arion/projects/GENECAD/Robert/DOE/PanUKBB/Download/download_links_{i+1}.txt', index=False, header=False)


In [ ]:
pu_var = pd.read_csv('./PanUKBB/full_variant_qc_metrics_filtered.bim', sep='\t', header=None)[[1,0,3,4,5]].set_axis(['id','chr','pos','ref','alt'],axis=1)
pu_conv = pd.read_csv('./PanUKBB/full_variant_qc_metrics_filtered.invr.txt', sep='\t')
pu_conv = pu_conv.drop('category',axis=1).set_axis(['id','chr','pos_38','pos'],axis=1)
pu_var = pu_var.merge(pu_conv)
pu_var['id_19'] = pu_var['chr'].astype(str) + '_' + pu_var['pos'].astype(str) + '_' + pu_var['ref'] + '_' + pu_var['alt']
pu_var['id_38'] = pu_var['chr'].astype(str) + '_' + pu_var['pos_38'].astype(str) + '_' + pu_var['ref'] + '_' + pu_var['alt']
pu_var[['id_19','id_38','chr','pos','ref','alt']].to_pickle('./PanUKBB/variant_conv.pkl')


## Finngen

In [ ]:
# Find relevant phenotypes
fgm = pd.read_csv('./Finngen/finngen_R12_manifest.tsv', sep='\t')
fg = pd.read_excel('./Finngen/finngen_R12_endpoints.xlsx')[['LEVEL','NAME','LONGNAME','HD_ICD_10','COD_ICD_10','version','PARENT']]
fg = fg.loc[fg['NAME'].isin(fgm['phenocode'])]
fg['ICD'] = fg['HD_ICD_10'].str.replace('.','')

a = fg.loc[fg['ICD'].isin(icd['ICD'])]
b = fg.loc[fg['LONGNAME'].isin(custom['finngen'])]
c = fg.loc[(~fg['ICD'].isin(icd['ICD'])) & (fg['HD_ICD_10'].notna())]
c['ICD_3c'] = c['ICD'].str[:3]
c = c.loc[c['ICD_3c'].isin(icd['ICD'])]
c_inc = pd.read_excel('./Finngen/bar_pheno.xlsx')
c_inc = c_inc.loc[c_inc['Include'] == 'Y']['NAME']
c = c.loc[(~c['HD_ICD_10'].str.contains('|', regex=False)) | (c['NAME'].isin(c_inc))]

fg = pd.concat([a,b,c])
fgm = fgm.loc[fgm['phenocode'].isin(fg['NAME'])]
fgm['path_https'].to_csv('./Finngen/download_links.txt', index=False, header=False)

for i, chunk in enumerate(range(0, len(fgm), 100)):
    fgm['path_https'].iloc[chunk:chunk+100].to_csv(f'/sc/arion/projects/GENECAD/Robert/DOE/Finngen/Download/download_links_{i+1}.txt', index=False, header=False)
    


In [ ]:
df = pd.read_csv('./Finngen/Annotations/var_anno_filt.txt', sep='\t')
df = df.loc[df['INFO'] >= 0.8]
df['max_af'] = df[['EXOME_AF_fin','GENOME_AF_fin']].max(axis=1)
df.loc[df['most_severe'].str.contains('stop_lost|start_lost|transcript_amplification|inframe_insertion|inframe_deletion|missense_variant|protein_altering_variant',na=False),'snp_type'] = 'Missense'
df.loc[df['most_severe'].str.contains('transcript_ablation|splice_acceptor_variant|splice_donor_variant|stop_gained|frameshift_variant',na=False),'snp_type'] = 'PTV'

# Save common variants
cl = df.loc[df['max_af'] >= 0.01]
cl = cl[['#variant','chr','pos','ref','alt']].rename({'#variant':'id'},axis=1)
cl.to_csv('./Finngen/Annotations/common_var.tsv', sep='\t', index=False)

# Save rare variants as VCF
ml = df.loc[(df['max_af'] < 0.01) & (df['snp_type'].notna())]
ml = ml[['chr','pos','#variant','ref','alt']].rename({'#variant':'id'},axis=1)
ml.loc[ml['chr'] == 23, 'chr'] = 'X'

ml['qual'] = '.'
ml['filter'] = '.'
ml['info'] = '.'

vcf_columns = ['chr', 'pos', 'id', 'ref', 'alt', 'qual', 'filter', 'info']
vcf_df = ml[vcf_columns]
vcf_df = vcf_df.sort_values(['chr','pos','ref','alt'])

vcf_header = """##fileformat=VCFv4.2
##source=CustomGeneratedVCF
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO
"""
with open('./Finngen/Annotations/sv_unique.vcf', 'w') as f:
    f.write(vcf_header)
    vcf_df.to_csv(f, sep='\t', index=False, header=False)


### Run VEP for rare missense and LOF variants

In [ ]:
# Process VEP outputs
vcf = pd.read_csv(f'./Finngen/Annotations/sv_unique_vep_filter.vcf', sep='\t', skiprows=53)
vcf = vcf.set_axis(['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'],axis=1).drop(['QUAL','FILTER'],axis=1)
vcf = vcf.assign(INFO=vcf['INFO'].str.split(',')).explode('INFO')
vcf['INFO'] = vcf['INFO'].str.split('|')

column_names = [
    'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature',
    'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position',
    'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'REF_ALLELE',
    'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT',
    'Aloft_pred', 'AlphaMissense_pred', 'AlphaMissense_rankscore', 'AlphaMissense_score',
    'BayesDel_addAF_pred', 'BayesDel_noAF_pred', 'CADD_raw', 'CADD_raw_rankscore',
    'ClinPred_pred', 'DANN_rankscore', 'DANN_score', 'DEOGEN2_pred', 'Eigen-PC-raw_coding',
    'Eigen-PC-raw_coding_rankscore', 'Eigen-raw_coding', 'Eigen-raw_coding_rankscore',
    'FATHMM_pred', 'LIST-S2_pred', 'LRT_pred', 'M-CAP_pred', 'MPC_rankscore', 'MPC_score',
    'MVP_rankscore', 'MVP_score', 'MetaLR_pred', 'MetaSVM_pred', 'MutPred_rankscore',
    'MutPred_score', 'MutationAssessor_pred', 'MutationTaster_pred', 'PROVEAN_pred',
    'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 'PrimateAI_pred', 'REVEL_rankscore',
    'REVEL_score', 'SIFT4G_pred', 'SIFT_pred', 'VEST4_rankscore', 'VEST4_score',
    'fathmm-MKL_coding_pred', 'fathmm-MKL_coding_rankscore', 'fathmm-XF_coding_pred',
    'fathmm-XF_coding_rankscore', 'gnomAD_exomes_AF', 'gnomAD_exomes_POPMAX_AF',
    'gnomAD_genomes_AF', 'gnomAD_genomes_POPMAX_AF'
]

vcf[column_names] = pd.DataFrame(vcf['INFO'].tolist(), index=vcf.index)
vcf = vcf.drop(columns=['INFO'])
vcf = vcf.drop(['Allele','Feature_type','Feature','BIOTYPE','EXON','INTRON','HGVSc','HGVSp',
                'cDNA_position','CDS_position','Protein_position','Amino_acids','Codons','Existing_variation',
                'REF_ALLELE','DISTANCE','STRAND','FLAGS','SYMBOL_SOURCE','HGNC_ID',
                'gnomAD_genomes_AF','gnomAD_genomes_POPMAX_AF'],axis=1)

vcf.loc[vcf['Consequence'].str.contains('stop_lost|start_lost|transcript_amplification|inframe_insertion|inframe_deletion|missense_variant|protein_altering_variant',na=False),'snp_type'] = 'Missense'
vcf.loc[vcf['Consequence'].str.contains('transcript_ablation|splice_acceptor_variant|splice_donor_variant|stop_gained|frameshift_variant',na=False),'snp_type'] = 'PTV'
vcf = vcf.loc[vcf['IMPACT'] != 'MODIFIER']
vcf = vcf.loc[vcf['snp_type'].notna()]

#####

qualitative_tools = ['SIFT_pred', 'SIFT4G_pred', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 
                     'LRT_pred', 'MutationTaster_pred', 'FATHMM_pred', 'PROVEAN_pred', 
                     'MetaSVM_pred', 'MetaLR_pred', 'M-CAP_pred', 'PrimateAI_pred', 
                     'DEOGEN2_pred', 'BayesDel_addAF_pred', 'BayesDel_noAF_pred', 
                     'ClinPred_pred', 'LIST-S2_pred', 'fathmm-MKL_coding_pred', 
                     'fathmm-XF_coding_pred', 'MutationAssessor_pred', 'Aloft_pred']

quantitative_tools = ['VEST4_rankscore', 'REVEL_rankscore', 'MutPred_rankscore', 
                      'MVP_rankscore', 'MPC_rankscore', 'DANN_rankscore', 'CADD_raw_rankscore', 
                      'Eigen-raw_coding_rankscore', 'Eigen-PC-raw_coding_rankscore']


def calculate_score(row):
    score = 0
    count = 0
    
    # Qualitative scoring
    for tool in qualitative_tools:
        if tool in row and row[tool] != '':
            if tool == 'MutationAssessor_pred' and row[tool] == 'H':
                score += 1
            elif tool == 'Aloft_pred' and row[tool] in ['R', 'D']:
                score += 1
            elif tool == 'MutationTaster_pred' and 'D' in str(row[tool]):
                score += 1
            elif row[tool] == 'D':
                score += 1
            count += 1
    
    # Quantitative scoring
    for tool in quantitative_tools:
        if tool in row and row[tool] != '':
            try:
                if float(row[tool]) > 0.9:
                    score += 1
                count += 1
            except ValueError:
                pass  # Ignore non-numeric values

    return score, count

# Apply missense scoring function
vcf[['total_score', 'non_missing_count']] = vcf.apply(lambda row: pd.Series(calculate_score(row)), axis=1)

vcf['Pred5'] = 0
vcf.loc[vcf['SIFT_pred'].str.contains('D',na=False), 'Pred5'] += 1
vcf.loc[vcf['LRT_pred'].str.contains('D',na=False), 'Pred5'] += 1
vcf.loc[vcf['MutationTaster_pred'].str.contains('A|D',na=False), 'Pred5'] += 1
vcf.loc[vcf['Polyphen2_HDIV_pred'].str.contains('D',na=False), 'Pred5'] += 1
vcf.loc[vcf['Polyphen2_HVAR_pred'].str.contains('D',na=False), 'Pred5'] += 1

#####

vcf = vcf[['ID','CHROM','POS','REF','ALT','snp_type','Consequence','IMPACT','SYMBOL',
           'total_score','non_missing_count','Pred5',
           'AlphaMissense_pred','AlphaMissense_rankscore','AlphaMissense_score',
           'REVEL_rankscore','REVEL_score',
           'gnomAD_exomes_AF','gnomAD_exomes_POPMAX_AF']]
vcf.to_csv(f'./Finngen/Annotations/sv_unique_vep_filter_cleaned.vcf', sep='\t', index=None)


### Run LOFTEE

In [ ]:
vcf = pd.read_csv('./Finngen/Annotations/sv_unique_vep_filter_cleaned.vcf', sep='\t')
vcf['ID'].drop_duplicates().to_csv('./Finngen/Annotations/rare_missense_lof.txt', sep='\t', index=False, header=None)


In [ ]:
vcf = pd.read_csv(f'./Finngen/Annotations/sv_unique_loftee_filter.vcf', sep='\t', skiprows=53)
vcf = vcf.set_axis(['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'],axis=1).drop(['QUAL','FILTER'],axis=1)
vcf = vcf.assign(INFO=vcf['INFO'].str.split(',')).explode('INFO')
vcf['INFO'] = vcf['INFO'].str.split('|')

column_names = [
    'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature',
    'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position',
    'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'REF_ALLELE',
    'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT',
    'LoF','LoF_filter','LoF_flags','LoF_info'
]

vcf[column_names] = pd.DataFrame(vcf['INFO'].tolist(), index=vcf.index)
vcf = vcf.drop(columns=['INFO'])
vcf.loc[vcf['Consequence'].str.contains('stop_lost|start_lost|transcript_amplification|inframe_insertion|inframe_deletion|missense_variant|protein_altering_variant',na=False),'snp_type'] = 'Missense'
vcf.loc[vcf['Consequence'].str.contains('transcript_ablation|splice_acceptor_variant|splice_donor_variant|stop_gained|frameshift_variant',na=False),'snp_type'] = 'PTV'
vcf = vcf.loc[vcf['IMPACT'] != 'MODIFIER']
vcf = vcf[['ID','Consequence','SYMBOL','LoF','LoF_filter','LoF_flags','LoF_info']]
vcf.to_pickle('./Finngen/Annotations/vep_loftee.pkl')


### Combining VEP outputs

In [20]:
vep = pd.read_csv('./Finngen/Annotations/sv_unique_vep_filter_cleaned.vcf', sep='\t')
loftee = pd.read_pickle('./Finngen/Annotations/vep_loftee.pkl')
vep = vep.merge(loftee, how='left')
vep['missense_score'] = vep['total_score']/vep['non_missing_count']
vep = vep.drop(['LoF_filter','LoF_info'],axis=1)

gof = pd.read_pickle('./LoGoFunc/all_predictions.pkl')
gof = gof.rename({'#CHROM':'CHROM'},axis=1).drop(['ID','prediction','LoGoFunc_LOF','SYMBOL'],axis=1)
vep = vep.merge(gof, how='left')

lof = pd.read_pickle('./LoGoFunc/LOF_predictions.pkl').rename({'#CHROM':'CHROM'},axis=1).drop(['ID','SYMBOL'],axis=1)
vep = vep.merge(lof, how='left')

vep['annotation'] = '5_other_missense'
vep.loc[(vep['non_missing_count'] > 7) & (vep['missense_score'] > 0.5), 'annotation'] = '4_other_missense_0.5'
vep.loc[vep['LoGoFunc_GOF'] > 0, 'annotation'] = '3_gof'
vep.loc[vep['LoGoFunc_LOF'] > 0, 'annotation'] = '2_lof_lc'
vep.loc[(vep['LoF'] == 'LC'), 'annotation'] = '2_lof_lc'
vep.loc[(vep['LoF'] == 'HC'), 'annotation'] = '1_lof_hc'

vep = vep.sort_values(['annotation','AlphaMissense_score'], ascending=[True,False]).drop_duplicates('ID')
vep = vep[['ID','SYMBOL','CHROM','POS','REF','ALT','snp_type','Consequence','annotation',
           'non_missing_count','missense_score']]

vep.to_pickle('./Finngen/Annotations/vep_cleaned.pkl')


/tmp/ipykernel_3075857/1911103171.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  vep = pd.read_csv('./Finngen/Annotations/sv_unique_vep_filter_cleaned.vcf', sep='\t')


### Subset common and rare variants

In [ ]:
bsub -q premium -P acc_GENECAD -W 23:59 -n 4 -o run.out -e run.error -R "span[hosts=1]" -R "rusage[mem=1GB]" \
'for f in /sc/arion/projects/GENECAD/Robert/DOE/Finngen/Processed/*.sig; do awk -F"\t" '\''NR==FNR{a[$1];next}FNR==1{OFS="\t";print "id","chr","pos","ref","alt","log10p","beta","se"}{id=$1":"$2":"$3":"$4;if(id in a)print id,$1,$2,$3,$4,$8,$9,$10}'\'' /sc/arion/projects/GENECAD/Robert/DOE/eQTL/finngen_unique_variants.tsv "$f" > "/sc/arion/projects/GENECAD/Robert/DOE/Finngen/Common/$(basename "$f")"; done'

bsub -q premium -P acc_GENECAD -W 23:59 -n 4 -o run.out -e run.error -R "span[hosts=1]" -R "rusage[mem=1GB]" \
'for f in /sc/arion/projects/GENECAD/Robert/DOE/Finngen/Processed/*.sig; do awk -F"\t" '\''NR==FNR{a[$1];next}FNR==1{OFS="\t";print "id","chr","pos","ref","alt","log10p","beta","se"}{id=$1":"$2":"$3":"$4;if(id in a)print id,$1,$2,$3,$4,$8,$9,$10}'\'' /sc/arion/projects/GENECAD/Robert/DOE/Finngen/Annotations/rare_missense_lof.txt "$f" > "/sc/arion/projects/GENECAD/Robert/DOE/Finngen/Rare/$(basename "$f")"; done'

(echo -e "id\tchr\tpos\tref\talt\tlog10p\tbeta\tse\tpheno"
for f in /sc/arion/projects/GENECAD/Robert/DOE/Finngen/Rare/finngen_R12_*.sig; do
  pheno=$(basename "$f" .sig | sed 's/finngen_R12_//')
  tail -n +2 "$f" | awk -v p="$pheno" 'BEGIN{OFS="\t"}{print $1,$2,$3,$4,$5,$6,$7,$8,p}'
done) > /sc/arion/projects/GENECAD/Robert/DOE/Finngen/rare_missense_lof.tsv
